In [ ]:
#create a new dataset containing latitude, longitude, time, temperature, RH, and 5 thermal comfort indices
import xarray as xr
import numpy as np
import math
from netCDF4 import Dataset

# Function to calculate relative humidity
def calculate_relative_humidity(T, Td):
    es = 6.112 * np.exp((17.67 * (Td-273.15)) / ((Td-273.15) + 243.04))  # Saturation vapor pressure
    e = 6.112 * np.exp((17.67 * (T-273.15)) / ((T-273.15) + 243.04))    # Vapor pressure
    RH = (es / e) * 100  # Relative humidity (%)
    return RH

# Function to calculate resultant wind speed
def calculate_resultant_wind_speed(u, v):
    ws = np.sqrt(u**2 + v**2)  # Wind speed
    return ws

# Function to calculate effective temperature (ET)
def calculate_effective_temperature(T, RH, ws):
    ET = 37- ((37 - (T-273.15))/(0.68 - 0.0014 * RH + (1/(1.76 + 1.4 * ws**0.75)))) - 0.29 * (T-273.15) * (1 - 0.01 * RH)
    return ET

# Function to calculate heat index (HI)
def calculate_heat_index(T, RH):
    HI = (-8.784695 + 1.61139411*(T-273.15) + 2.338549*RH - 0.14611605*(T-273.15)*RH - 0.012308094*(T-273.15)**2 - 0.016424828*RH**2 + 0.002211732*(T-273.15)**2*RH + 0.00072546*(T-273.15)*RH**2 - 0.000003582*(T-273.15)**2*RH**2)
    return HI

# Function to calculate humidex
def calculate_humidex(T, RH):
    e = 6.11 * (10**((7.5 * (T-273.15))/(237.7 + (T-273.15)))) * (RH/100)    
    h = (5/9) * (e - 10.0)
    Humidex = (T-273.15) + h
    return Humidex

# Function to calculate wet bulb globe temperature index (WBGT)
def calculate_wbgt(T, RH):
    e = 6.11 * (10**((7.5 * (T-273.15))/(237.7 + (T-273.15)))) * (RH/100)
    WBGT = 0.567*(T-273.15) + 0.393*e + 3.94
    return WBGT

# Function to calculate discomfort index
def calculate_discomfort_index(T, RH):
    DI = (T-273.15) - (0.55 - 0.0055 * RH) * ((T-273.15) - 14.5)
    return DI

# Path to input NetCDF file
input_nc_file = '/media/Storage/ThermalComfortPaper_Jennyfer/ERA5_ReanalysisData/2023/adaptor.mars.internal-1720658446.9816003-7167-20-6779c5ff-542d-4fbc-8551-30bc63891e22.nc'

# Path to output NetCDF file
output_nc_file = '/media/Storage/ThermalComfortPaper_Jennyfer/ERA5_ReanalysisData/2023/AllCities_Updated'
# Open the input NetCDF file
ds = xr.open_dataset(input_nc_file)

# Extract variables
latitudes = ds['latitude'].values
longitudes = ds['longitude'].values
times = ds['time'].values
T = ds['t2m'].values  # Dry bulb temperature (in Kelvin)
Td = ds['d2m'].values  # Dew point temperature (in Kelvin)
u = ds['u10'].values  # U direction of velocity
v = ds['v10'].values  # V direction of velocity

# Calculate relative humidity
RH = calculate_relative_humidity(T, Td)

# Calculate resultant wind speed
WS = calculate_resultant_wind_speed(u, v)

# Calculate effective temperature (ET)
ET = calculate_effective_temperature(T, RH, WS)

# Calculate heat index (HI)
HI = calculate_heat_index(T, RH)

# Calculate humidex
Humidex = calculate_humidex(T, RH)

# Calculate wet bulb globe temperature index (WBGT)
WBGT = calculate_wbgt(T, RH)

# Calculate discomfort index (DI)
DI = calculate_discomfort_index(T, RH)

# Create a new xarray Dataset for output
output_ds = xr.Dataset({
    'latitude': (('latitude'), latitudes),
    'longitude': (('longitude'), longitudes),
    'time': (('time'), times),
    'T': (('time', 'latitude', 'longitude'), T),
    'RH': (('time', 'latitude', 'longitude'), RH),
    'WS': (('time', 'latitude', 'longitude'), WS),
    'ET': (('time', 'latitude', 'longitude'), ET),
    'HI': (('time', 'latitude', 'longitude'), HI),
    'HD': (('time', 'latitude', 'longitude'), Humidex),
    'WBGT': (('time', 'latitude', 'longitude'), WBGT),
    'DI': (('time', 'latitude', 'longitude'), DI)
})


# Save the new dataset to a new NetCDF file
output_ds.to_netcdf(output_nc_file, format='NETCDF4', engine='netcdf4')

print(f"Processed data saved to: {output_nc_file}")

# Close input dataset
ds.close()


In [ ]:
#create a new dataset containing latitude, longitude, time, wet bulb temperature (for the trend maps)
import xarray as xr
import numpy as np
import math
from netCDF4 import Dataset

# Function to calculate wet bulb temperature
def calculate_Tw(T, RH):
    term1 = (T-273.15) * np.arctan(0.152 * np.sqrt(RH + 8.3136))
    term2 = np.arctan((T-273.15) + RH)
    term3 = np.arctan(RH - 1.6763)
    term4 = 0.00391838 * (RH ** 1.5) * np.arctan(0.0231 * RH)
    constant = 4.686
    
    Tw = term1 + term2 - term3 + term4 - constant
    return Tw


# Path to input NetCDF file
input_nc_file = '/media/Storage/ThermalComfortPaper_Jennyfer/ERA5_ReanalysisData/AverageMonths/AverageMonths_Updated.nc'

# Path to output NetCDF file
output_nc_file = '/media/Storage/ThermalComfortPaper_Jennyfer/ERA5_ReanalysisData/AverageMonths/WBT'
# Open the input NetCDF file
ds = xr.open_dataset(input_nc_file)

# Extract variables
latitudes = ds['latitude'].values
longitudes = ds['longitude'].values
times = ds['time'].values
T = ds['T'].values  # Dry bulb temperature (in Kelvin)
RH = ds['RH'].values  # Relative humidity (%)


# Calculate Tw
T_w = calculate_Tw(T, RH)
# Create a new xarray Dataset for output
output_ds = xr.Dataset({
    'latitude': (('latitude'), latitudes),
    'longitude': (('longitude'), longitudes),
    'time': (('time'), times),
    'Tw': (('time', 'latitude', 'longitude'), T_w)
})


# Save the new dataset to a new NetCDF file
output_ds.to_netcdf(output_nc_file, format='NETCDF4', engine='netcdf4')

print(f"Processed data saved to: {output_nc_file}")

# Close input dataset
ds.close()


In [ ]:
#To get the temperature or RH of a certain city averaged for the 43-year period.
import xarray as xr
import numpy as np

# Load the NetCDF file
file_path = '/media/Storage/ThermalComfortPaper_Jennyfer/ERA5_ReanalysisData/AverageMonths/AverageMonths_Updated.nc'
ds = xr.open_dataset(file_path)

# Riyadh's geographical coordinates
riyadh_lat = 39.9334
riyadh_lon = 32.8597

# Find the nearest latitude and longitude indices
lat_idx = np.abs(ds['latitude'] - riyadh_lat).argmin().item()
lon_idx = np.abs(ds['longitude'] - riyadh_lon).argmin().item()

# Select the temperature data for the summer months (June, July, and August) from 1980 to 2023
summer_months = ds['time.month'].isin([5, 6, 7, 8, 9]) #from May to September
temperature_summer = ds['T'].sel(
    time=summer_months,
    latitude=ds['latitude'][lat_idx],
    longitude=ds['longitude'][lon_idx]
)

# Group by year and calculate the mean temperature for each summer
summer_avg_by_year = temperature_summer.groupby('time.year').mean(dim='time')

# Calculate the average summer temperature over the entire period from 1980 to 2023
average_summer_temperature_1980_2023 = summer_avg_by_year.mean(dim='year')

# Convert the temperature from Kelvin to Celsius
average_summer_temperature_celsius = average_summer_temperature_1980_2023 - 273.15

# Print the average summer temperature
print(f"Average summer temperature in Riyadh from 1980 to 2023: {average_summer_temperature_celsius.values:.2f} degree C")


In [ ]:
#To get the temperature and RH of the cities seasonally and annualy for each decade (for the table in the appendix)
import xarray as xr
import numpy as np

# Load the NetCDF file
file_path = '/media/Storage/ThermalComfortPaper_Jennyfer/ERA5_ReanalysisData/AverageMonths/AverageMonths_Updated.nc'
ds = xr.open_dataset(file_path)

# Riyadh's geographical coordinates
riyadh_lat =41.0082
riyadh_lon =28.9784

# Find the nearest latitude and longitude indices
lat_idx = np.abs(ds['latitude'] - riyadh_lat).argmin().item()
lon_idx = np.abs(ds['longitude'] - riyadh_lon).argmin().item()

# Define the year ranges
year_ranges = {
    '1980-1990': ('1980-01-01', '1989-12-31'),
    '1990-2000': ('1990-01-01', '1999-12-31'),
    '2000-2010': ('2000-01-01', '2009-12-31'),
    '2010-2020': ('2010-01-01', '2019-12-31'),
    '2020-2023': ('2020-01-01', '2023-12-31')
}

# Dictionary to store average summer temperatures for each year range
average_temperatures = {}

# Loop through each year range and calculate the average summer temperature
for period, (start_date, end_date) in year_ranges.items():
    # Select the temperature data for the specific time range
    time_range = ds['time'].sel(time=slice(start_date, end_date))
    
    # Further filter for the summer months (June, July, and August)
    summer_months = time_range['time.month'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]) # 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12
    temperature_summer = ds['RH'].sel(
        time=time_range['time'][summer_months],
        latitude=ds['latitude'][lat_idx],
        longitude=ds['longitude'][lon_idx]
    )
    
    # Group by year and calculate the mean temperature for each summer
    summer_avg_by_year = temperature_summer.groupby('time.year').mean(dim='time')
    
    # Calculate the average summer temperature over the specified period
    average_temperature = summer_avg_by_year.mean(dim='year')
    
    # Convert the temperature from Kelvin to Celsius
    average_temperature_celsius = average_temperature #- 273.15
    
    # Store the result in the dictionary
    average_temperatures[period] = average_temperature_celsius.values

# Print the average summer temperatures for each year range
for period, avg_temp in average_temperatures.items():
    print(f"Average summer temperature in Riyadh from {period}: {avg_temp:.2f} %")


In [ ]:
#to find the summation of the number of days during the summer period over the total land area during which the ET index exceeds 23 degrees meaning hot, very hot, or sweltering sensations.
import numpy as np
import xarray as xr

# Load the .nc file using xarray
file_path = "/media/Storage/ThermalComfortPaper_Jennyfer/ERA5_ReanalysisData/2023/AllCities_Updated"
data = xr.open_dataset(file_path)

# Extract necessary variables
latitude = data['latitude'].values
longitude = data['longitude'].values
time = data['time']
ET = data['ET'].values  # Replace 'ET' with the actual variable name if different

# Define the months of interest (May to September)
months_of_interest = [5, 6, 7, 8, 9]

# Create a mask for land points based on a threshold or a land mask file if available
# Assuming ET values are NaN over the ocean, indicating ocean points.
land_mask = ~np.isnan(ET[0, :, :])

# Initialize a counter for days where ET > 23 for more than 8 hours in a row
exceedance_days = np.zeros((len(latitude), len(longitude)), dtype=int)

# Convert the time variable to a pandas datetime index for easier month filtering
time_index = time.to_index()

# Iterate over each grid point and calculate the days
for lat_idx in range(len(latitude)):
    for lon_idx in range(len(longitude)):
        if land_mask[lat_idx, lon_idx]:
            # Extract time series for the current grid point
            et_series = ET[:, lat_idx, lon_idx]
           
            # Filter data for the months of interest
            et_series_filtered = et_series[np.isin(time_index.month, months_of_interest)]
            time_filtered = time_index[np.isin(time_index.month, months_of_interest)]
           
            # Reshape into days (24 hours each)
            daily_et_series = et_series_filtered.reshape(-1, 24)
           
            # Check for days where ET > 23 for more than 8 hours consecutively
            for day in daily_et_series:
                consecutive_hours = np.sum(day > 23)
                if consecutive_hours >= 8:
                    exceedance_days[lat_idx, lon_idx] += 1

# Sum up all the exceedance days to get one total number
total_exceedance_days = np.sum(exceedance_days)

# Print the total number of exceedance days across all land points
print("Total number of exceedance days where ET > 23 for more than 8 hours in 2023:", total_exceedance_days)


In [ ]:
#WBGT
#to find the summation of the number of days during the summer period over the total land area during which the WBGT index exceeds 24 degrees meaning hot, very hot, or sweltering sensations.
import numpy as np
import xarray as xr

# Load the .nc file using xarray
file_path = "/media/Storage/ThermalComfortPaper_Jennyfer/ERA5_ReanalysisData/2023/AllCities_Updated"
data = xr.open_dataset(file_path)

# Extract necessary variables
latitude = data['latitude'].values
longitude = data['longitude'].values
time = data['time']
WBGT = data['WBGT'].values 

# Define the months of interest (May to September)
months_of_interest = [5, 6, 7, 8, 9]

# Create a mask for land points based on a threshold or a land mask file if available 
# Assuming WBGT values are NaN over the ocean, indicating ocean points.
land_mask = ~np.isnan(ET[0, :, :])

# Initialize a counter for days where WBGT > 24 for more than 8 hours in a row
exceedance_days = np.zeros((len(latitude), len(longitude)), dtype=int)

# Convert the time variable to a pandas datetime index for easier month filtering
time_index = time.to_index()

# Iterate over each grid point and calculate the days
for lat_idx in range(len(latitude)):
    for lon_idx in range(len(longitude)):
        if land_mask[lat_idx, lon_idx]:
            # Extract time series for the current grid point
            wbgt_series = WBGT[:, lat_idx, lon_idx]
           
            # Filter data for the months of interest
            wbgt_series_filtered = wbgt_series[np.isin(time_index.month, months_of_interest)]
            time_filtered = time_index[np.isin(time_index.month, months_of_interest)]
           
            # Reshape into days (24 hours each)
            daily_wbgt_series = wbgt_series_filtered.reshape(-1, 24)
           
            # Check for days where WBGT > 24 for more than 8 hours consecutively
            for day in daily_wbgt_series:
                consecutive_hours = np.sum(day > 24)
                if consecutive_hours >= 8:
                    exceedance_days[lat_idx, lon_idx] += 1

# Sum up all the exceedance days to get one total number
total_exceedance_days = np.sum(exceedance_days)

# Print the total number of exceedance days across all land points
print("Total number of exceedance days where WBGT > 24 for more than 8 hours in 2023:", total_exceedance_days)

In [1256]:
#hourly analysis for one nc file
import xarray as xr
import pandas as pd

# Load the ERA5 dataset
ds = xr.open_dataset('/media/Storage/ThermalComfortPaper_Jennyfer/ERA5-UpdatedData/1982/Tehran_updated.nc')

# Ensure time is in datetime format
ds['time'] = pd.to_datetime(ds['time'].values)

# Filter for summer months (May to September)
summer = ds.sel(time=ds['time'].dt.month.isin([5, 6, 7, 8, 9]))

# Define daytime (6 AM to 6 PM) and nighttime (6 PM to 6 AM)
daytime = summer.sel(time=summer['time'].dt.hour.isin(range(6, 18)))
nighttime = summer.sel(time=~summer['time'].dt.hour.isin(range(6, 18)))

# Calculate averages for daytime and nighttime
average_daytime = daytime[['T', 'RH', 'WS']].mean(dim='time')
average_nighttime = nighttime[['T', 'RH', 'WS']].mean(dim='time')

# Print the results
print("Daytime Averages:")
print(average_daytime)

print("\nNighttime Averages:")
print(average_nighttime)

# Optionally, save results to a NetCDF or CSV
# average_daytime.to_netcdf('daytime_averages.nc')
# average_nighttime.to_netcdf('nighttime_averages.nc')
# average_daytime.to_dataframe().to_csv('daytime_averages.csv')
# average_nighttime.to_dataframe().to_csv('nighttime_averages.csv')



Daytime Averages:
<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1)
Coordinates:
  * latitude   (latitude) float32 35.75
  * longitude  (longitude) float32 51.5
Data variables:
    T          (latitude, longitude) float32 297.1
    RH         (latitude, longitude) float32 25.73
    WS         (latitude, longitude) float32 2.381

Nighttime Averages:
<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1)
Coordinates:
  * latitude   (latitude) float32 35.75
  * longitude  (longitude) float32 51.5
Data variables:
    T          (latitude, longitude) float32 291.2
    RH         (latitude, longitude) float32 37.91
    WS         (latitude, longitude) float32 1.183


###### 